# EfficientDet Transfer Learning on TACO Dataset

This notebook demonstrates how to perform transfer learning using EfficientDet-D0 on the TACO (Trash Annotations in Context) dataset.

## 0. Setup and Requirements

First, let's install the required packages.

In [1]:
!pip install pycocotools numpy opencv-python tqdm tensorboard tensorboardX pyyaml webcolors matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.9 MB/s eta 0:00:00


## 1. Import Dependencies and Set Up Environment

In [6]:
import os
import sys
import torch
from torch.backends import cudnn

# Add the project directory to the Python path
if "Yet-Another-EfficientDet-Pytorch" not in os.getcwd():
    !git clone --depth 1 https://github.com/markintoshplus/Yet-Another-EfficientDet-Pytorch.git
    os.chdir('Yet-Another-EfficientDet-Pytorch')
    sys.path.append('.')
else:
    !git pull

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0 (from 0)
Unpacking objects: 100% (6/6), 542 bytes | 36.00 KiB/s, done.
From https://github.com/markintoshplus/Yet-Another-EfficientDet-Pytorch
   4a65c28..a0fdec2  master     -> origin/master
Updating 4a65c28..a0fdec2
Fast-forward
 efficientdet/dataset.py                | 2 +-
 tutorial/train_birdview_vehicles.ipynb | 4 ----
 2 files changed, 1 insertion(+), 5 deletions(-)


## 2. Prepare Dataset and Weights

Let's verify the locations of the TACO dataset, pre-trained weights, and the project YAML file.

In [7]:
# Verify TACO dataset location
print("TACO dataset contents:")
print(os.listdir('datasets/taco'))

# Verify annotations
print("\nAnnotations:")
print(os.listdir('datasets/taco/annotations'))

# Display a sample of image files in train folder
print("\nSample of training images:")
print(os.listdir('datasets/taco/train')[:5])

TACO dataset contents:
['val', 'test', 'annotations', 'train']

Annotations:
['val_annotations.coco.json', 'test_annotations.coco.json', 'train_annotations.coco.json']

Sample of training images:
['batch_5000019_JPG.rf.cdd805cb4d7442374369fc4490bc011e.jpg', 'batch_14000013_jpg.rf.92b58a620abd39c5271fd5e4e889b325.jpg', 'batch_9000063_jpg.rf.5a8cb241c0f4c32dcb6b5d2ae09a7097.jpg', 'batch_4000061_JPG.rf.8172b5b335e2bc90cb5d0f1c0cb2d877.jpg', 'batch_3IMG_4919_JPG.rf.41a41ac787b91b783cef67d54f3d888b.jpg']


## 3. Training

We'll perform transfer learning in two steps:
1. Train only the head of the network
2. Fine-tune the entire model

In [8]:
# Train head only (transfer learning)
!python train.py -c 0 -p taco --head_only True --lr 1e-3 --batch_size 16 --load_weights pre-trained_weights/efficientdet-d0.pth --num_epochs 50 --save_interval 5

loading annotations into memory...
Done (t=0.04s)
creating index...
index created!
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
/content/Yet-Another-EfficientDet-Pytorch/train.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECU

In [ ]:
# Fine-tune the entire model
!python train.py -c 0 -p taco --head_only False --lr 1e-4 --batch_size 8 --load_weights logs/taco/efficientdet-d0_49_3250.pth --num_epochs 100 --save_interval 100

loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 12 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
loading annotations into memory...
Done (t=0.11s)
creating index...
index created!
/content/Yet-Another-EfficientDet-Pytorch/train.py:139: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECU

## 4. Evaluation

Now, let's evaluate the trained model on the TACO dataset.

In [ ]:
# Get the latest weight file
%cd logs/taco
weight_file = !ls -Art | grep efficientdet
%cd ../..

# Evaluate the model
!python coco_eval.py -c 0 -p taco -w "logs/taco/{weight_file[-1]}"

## 5. Visualization

Finally, let's visualize the model's predictions on a sample image from the TACO dataset.

In [ ]:
import torch
from torch.backends import cudnn
from backbone import EfficientDetBackbone
import cv2
import matplotlib.pyplot as plt
import numpy as np
import json

from efficientdet.utils import BBoxTransform, ClipBoxes
from utils.utils import preprocess, invert_affine, postprocess

compound_coef = 0
force_input_size = None  # set None to use default size
img_path = 'datasets/taco/val/batch_7000008_JPG.rf.14f30006571dfa762fd80920a1f333fe.jpg'

threshold = 0.2
iou_threshold = 0.2

use_cuda = True
use_float16 = False
cudnn.fastest = True
cudnn.benchmark = True

# Load TACO category names
with open('datasets/taco/annotations/test_annotations.coco.json') as f:
    annotations = json.load(f)
obj_list = [category['name'] for category in annotations['categories']]

# tf bilinear interpolation is different from any other's, just make do
input_sizes = [512, 640, 768, 896, 1024, 1280, 1280, 1536]
input_size = input_sizes[compound_coef] if force_input_size is None else force_input_size

model = EfficientDetBackbone(compound_coef=compound_coef, num_classes=len(obj_list),
                             ratios=[(1.0, 1.0), (1.4, 0.7), (0.7, 1.4)],
                             scales=[2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

model.load_state_dict(torch.load(f'weights/efficientdet-d{compound_coef}.pth'))
model.requires_grad_(False)
model.eval()

if use_cuda:
    model = model.cuda()
if use_float16:
    model = model.half()

ori_imgs, framed_imgs, framed_metas = preprocess(img_path, max_size=input_size)

if use_cuda:
    x = torch.stack([torch.from_numpy(fi).cuda() for fi in framed_imgs], 0)
else:
    x = torch.stack([torch.from_numpy(fi) for fi in framed_imgs], 0)

x = x.to(torch.float32 if not use_float16 else torch.float16).permute(0, 3, 1, 2)

with torch.no_grad():
    features, regression, classification, anchors = model(x)

    regressBoxes = BBoxTransform()
    clipBoxes = ClipBoxes()

    out = postprocess(x,
                      anchors, regression, classification,
                      regressBoxes, clipBoxes,
                      threshold, iou_threshold)

out = invert_affine(framed_metas, out)

for i in range(len(ori_imgs)):
    if len(out[i]['rois']) == 0:
        continue
    ori_imgs[i] = ori_imgs[i].copy()
    for j in range(len(out[i]['rois'])):
        (x1, y1, x2, y2) = out[i]['rois'][j].astype(np.int)
        cv2.rectangle(ori_imgs[i], (x1, y1), (x2, y2), (255, 255, 0), 2)
        obj = obj_list[out[i]['class_ids'][j]]
        score = float(out[i]['scores'][j])

        cv2.putText(ori_imgs[i], '{}, {:.3f}'.format(obj, score),
                    (x1, y1 + 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                    (255, 255, 0), 1)

plt.figure(figsize=(15, 15))
plt.imshow(ori_imgs[0])
plt.axis('off')
plt.show()